In [ ]:
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
from arxivrec.fetcher import fetch_recent_papers

In [ ]:
df = fetch_recent_papers(max_results=100)

In [ ]:
from arxivrec.encoder import TextEncoder

encoder = TextEncoder(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
import numpy as np

my_interest_embedding = encoder.encode(["I am interested in Retrieval-augmented generation (RAG)."])
content_embedding = encoder.encode(df['combined_text'].tolist())
print(my_interest_embedding.shape)
print(content_embedding.shape)
print(encoder.cosine_sim(my_interest_embedding, content_embedding))

top_k_indices = encoder.get_top_k_similar(my_interest_embedding, content_embedding, k=10)
print(top_k_indices)

In [ ]:
top_df = df.iloc[top_k_indices]

In [ ]:
from arxivrec.llm import fine_rank_with_llm

In [ ]:
refined_df = fine_rank_with_llm(top_df)

In [ ]:
refined_df